## NAを取り込むための変数の追加

前回のノートでは、欠損値を平均値、中央値、またはランダムな値で置き換える方法を学びました。言い換えると，平均値/中央値およびランダム・サンプル・インピュテーションについて学びました．これらの方法は，データが完全に無作為に欠落していることを前提としています（MCAR）．

値が無作為に欠落していない場合には，任意の値の組 み込みや分布の終わりの組込みなど，他の方法もあります．しかし、これらの代入法は変数の分布に大きな影響を与えるため、線形モデルには適していません。

**では、データがMCARではなく、線形モデルを使用したい場合はどうすればよいのでしょうか？

データが無作為に欠損していない場合、欠損したオブザベーションを平均値/中央値/最頻値で置き換え、さらにそれらの欠損したオブザベーションを **Missing Indicator** でフラグ** することが良いでしょう。欠落インジケータは、ある観測データが欠落しているか（1）、していないか（0）を示す、追加のバイナリ変数です。


### どのような変数に欠損インジケータを追加できますか？

### どのような変数に欠落インジケータを追加できますか？ 数値変数とカテゴリー変数の両方に欠落インジケータを追加できます。

#### 注意事項

欠落インジケータの追加は、決して単独では使用しません。逆に，数値変数の場合は平均値/中央値の入力，カテゴリー変数の場合は頻出カテゴリーの入力など，常に他の入力技術と一緒に使用されます。また，カテゴライズされた変数と数値変数の両方について，欠損インジケータを追加することと，ランダム・サンプルの入力を併用することもできる．

一般的な併用方法

- 平均値/中央値のインピュテーション + 欠損インジケータ (数値変数)
- 頻出カテゴリの入力 + 欠落インジケータ (カテゴリ変数)
- 無作為抽出によるインピュテーション + 欠損指標 (数値およびカテゴリー変数)

#### 前提条件

- データは無作為に欠損しない
- 欠損データは予測可能

### メリット

- 実装が容易
- 欠落データがある場合、その重要性を把握できる

### 制限事項

- 特徴空間の拡大
- NaNを除去するために、元の変数をインピュートする必要がある。

欠損インジケータを追加すると、データセット内で欠損値のある変数ごとに1つの変数が増えます。つまり、データセットに10個の特徴量があり、すべての特徴量に欠損値がある場合、欠損インジケータを追加すると、元の10個の特徴量に加えて、元の変数ごとに値が欠損しているかどうかを示す10個のバイナリ特徴量を加えた20個の特徴量を持つデータセットになります。これは、数十から数百の変数を持つデータセットでは問題にならないかもしれませんが、元のデータセットに数千の変数が含まれている場合、NAを示す追加の変数を作成すると、非常に大きなデータセットになってしまいます。

#### 重要

また、複数の変数で同じ観測値のデータが欠落する傾向があり、欠落した指標変数の多くが実際には似たり寄ったりになってしまうことがよくあります。

### 最終注意

一般的に、平均値/中央値/最頻値の組換えは、データが欠損しているオブザベーションを捕捉するための変数を追加して行われます。

どちらの方法も非常に簡単に実装できるため、データサイエンスのコンテストではトップの選択肢となります。例えば、KDD 2009カップの優勝ソリューションをご覧ください。["Winning the KDD Cup Orange Challenge with Ensemble Selection"](http://www.mtome.com/Publications/CiML/CiML-v3-book.pdf)。


## このデモでは

Ames House Price and Titanic Datasetsを使用します。

- データセットをダウンロードするには、このコースの**セクション1**にある**Datasets**の講義を参照してください。


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# データセットを分割する
from sklearn.model_selection import train_test_split

In [17]:
# デモのためにいくつかの変数を使ってTitanicデータセットをロードします。

data = pd.read_csv('/content/drive/MyDrive/特徴量工学/Section-04-Missing-Data-Imputation/titanic.csv', usecols=['age', 'fare', 'survived'])
data.head()

,survived,age,fare
0,1,29,211.3375
1,1,0.9167,151.55
2,0,2,151.55
3,0,30,151.55
4,0,25,151.55


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# NAの割合を見てみよう

data.isnull().mean()

survived    0.0
age         0.0
fare        0.0
dtype: float64

バイナリの欠損指標を追加するためには、必ずしもトレーニングセットから何かを学習する必要はありません。原理的には、元のデータセットでこれを行い、トレーニングとテストに分けることができます。しかし、私はこの方法をお勧めしません。
さらに、scikit-learnを使用して欠損インジケータを追加する場合、インジケータは訓練セットから、どの特徴をインピュートするか、つまり、どの特徴に対してバイナリ変数を追加する必要があるかを学習する必要があります。欠損指標のさまざまな実装については、今後のノートで詳しく説明します。今回は、バイナリのミッシングインジケータを手動で作成する方法を見てみましょう。


In [20]:
# トレーニングセットとテストセットに分けてみましょう

X_train, X_test, y_train, y_test = train_test_split(
    data[['age', 'fare']], # 予測変数
    data['survived'], #ターゲット
    test_size=0.3, # テストセットに含まれるオブの割合
    random_state=0) # 再現性を確保するための # 種付け

X_train.shape, X_test.shape


((916, 2), (393, 2))

In [21]:

# 訓練セットの欠損データを調べてみましょう
# パーセンテージは、データセット全体のものとかなり似ているはずです。
# データセット全体の


X_train.isnull().mean()

age     0.0
fare    0.0
dtype: float64

In [22]:
# 欠落している指標を追加する

# これは numpy の np.where を使うことで非常に簡単に行うことができます。
# そしてpandasの isnullを使うことで簡単にできます。


X_train['Age_NA'] = np.where(X_train['age'].isnull(), 1, 0)
X_test['Age_NA'] = np.where(X_test['age'].isnull(), 1, 0)

X_train.head()

,age,fare,Age_NA
501,13,19.5,0
588,4,23,0
402,30,13.8583,0
1193,?,7.725,0
686,22,7.725,0


In [23]:

# バイナリ変数の平均値は、元の変数の欠損値の割合と一致します。
# 元の変数の欠損値の数と一致する。

X_train['Age_NA'].mean()

0.0

In [24]:
# しかし、元の変数には、まだ欠損値があります。
# いずれかの手法で置き換える必要があります。
# we have learnt


X_train.isnull().mean()

age       0.0
fare      0.0
Age_NA    0.0
dtype: float64

In [25]:

X_train=X_train[(~X_train['age'].str.contains('?', regex=False))&(~
X_train['fare'].str.contains('?', regex=False).sum())]

In [26]:
# 例えばメディアンインピュテーション

median = X_train['age'].median()

X_train['age'] = X_train['age'].fillna(median)
X_test['age'] = X_test['age'].fillna(median)

# 欠損値がなくなったことを確認
X_train.isnull().mean()




age       0.0
fare      0.0
Age_NA    0.0
dtype: float64

### 家屋価格データセット

In [27]:
# 次のような変数を使用する予定です。
# あるものはカテゴリー、あるものは数値

cols_to_use = [
    'LotFrontage', 'MasVnrArea', # 数値
    'BsmtQual', 'FireplaceQu', # カテゴリー別
    'SalePrice' # ターゲット
]


In [28]:
# 家の値段のデータセットを読み込もう


data = pd.read_csv('/content/drive/MyDrive/特徴量工学/Section-04-Missing-Data-Imputation/HousePrice.csv', usecols=cols_to_use)
print(data.shape)
data.head()

(1460, 5)


,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,SalePrice
0,65.0,196.0,Gd,NaN,208500
1,80.0,0.0,Gd,TA,181500
2,68.0,162.0,Gd,TA,223500
3,60.0,0.0,TA,Gd,140000
4,84.0,350.0,Gd,TA,250000


In [29]:
# 欠損値のある変数を調べよう

data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
SalePrice      0.000000
dtype: float64

In [30]:
# トレーニングセットとテストセットに分けよう

X_train, X_test, y_train, y_test = train_test_split(data,
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

In [31]:
# 欠損指標を追加する関数を作ってみよう
# バイナリ変数



def missing_indicator(df, variable):    
    return np.where(df[variable].isnull(), 1, 0)

In [32]:
# 全ての変数をループさせて バイナリの欠落指標を追加しよう 
# 作成した関数で欠落した指標を追加しよう

for variable in cols_to_use:
    X_train[variable+'_NA'] = missing_indicator(X_train, variable)
    X_test[variable+'_NA'] = missing_indicator(X_test, variable)
    
X_train.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,SalePrice,LotFrontage_NA,MasVnrArea_NA,BsmtQual_NA,FireplaceQu_NA,SalePrice_NA
64,NaN,573.0,Gd,NaN,219500,1,0,0,1,0
682,NaN,0.0,Gd,Gd,173000,1,0,0,0,0
960,50.0,0.0,TA,NaN,116500,0,0,0,1,0
1384,60.0,0.0,TA,NaN,105000,0,0,0,1,0
1100,60.0,0.0,TA,NaN,60000,0,0,0,1,0


In [33]:
# それでは欠落した指標の平均値を評価してみましょう

# まず最初に、欠落している指標変数を 
# リスト内包
missing_ind = [col for col in X_train.columns if 'NA' in col]

# 平均値を計算
X_train[missing_ind].mean()



LotFrontage_NA    0.184932
MasVnrArea_NA     0.004892
BsmtQual_NA       0.023483
FireplaceQu_NA    0.467710
SalePrice_NA      0.000000
dtype: float64

In [34]:
# 欠損指標の平均値
# 欠損値の割合と一致する
# 元の変数の

X_train.isnull().mean()

LotFrontage       0.184932
MasVnrArea        0.004892
BsmtQual          0.023483
FireplaceQu       0.467710
SalePrice         0.000000
LotFrontage_NA    0.000000
MasVnrArea_NA     0.000000
BsmtQual_NA       0.000000
FireplaceQu_NA    0.000000
SalePrice_NA      0.000000
dtype: float64

In [35]:

# 欠損値を値で埋める関数を作ってみましょう．
# 以前のノートでも似たような関数を使ったことがあるので
# なので，よく知っていると思います


def impute_na(df, variable, value):
    return df[variable].fillna(value)

In [36]:

# NAを数値の中央値で埋めてみよう
# 変数
# 訓練セットを使って中央値を計算することを覚えておいてください
median = X_train['LotFrontage'].median()
X_train['LotFrontage'] = impute_na(X_train, 'LotFrontage', median)
X_test['LotFrontage'] = impute_na(X_test, 'LotFrontage', median)

median = X_train['MasVnrArea'].median()
X_train['MasVnrArea'] = impute_na(X_train, 'MasVnrArea', median)

X_test['MasVnrArea'] = impute_na(X_test, 'MasVnrArea', median)


# カテゴリー変数のNAを、最も頻度の高いカテゴリー（別名、中央値）に帰属させましょう。
# 最も頻度の高いカテゴリー（別名：最頻値）によって # 帰属させます。
# 最頻値は，訓練セットから学習する必要があります．

mode = X_train['BsmtQual'].mode()[0]
X_train['BsmtQual'] = impute_na(X_train, 'BsmtQual', mode)
X_test['BsmtQual'] = impute_na(X_test, 'BsmtQual', mode)

mode = X_train['FireplaceQu'].mode()[0]
X_train['FireplaceQu'] = impute_na(X_train, 'FireplaceQu', mode)
X_test['FireplaceQu'] = impute_na(X_test, 'FireplaceQu', mode)


In [37]:
# そして今度は、これ以上NAがないことを確認しよう
X_train.isnull().mean()

LotFrontage       0.0
MasVnrArea        0.0
BsmtQual          0.0
FireplaceQu       0.0
SalePrice         0.0
LotFrontage_NA    0.0
MasVnrArea_NA     0.0
BsmtQual_NA       0.0
FireplaceQu_NA    0.0
SalePrice_NA      0.0
dtype: float64


ご覧のように、元のデータセットに比べて特徴量が2倍になっています。元のデータセットには4つの変数がありましたが、前処理後のデータセットには8つの変数とターゲットが含まれています。

**以上が今回のデモの内容です。次回も楽しみにしていてくださいね。